In [1]:
import tensorwrap as tf
model = tf.nn.Sequential([
    tf.nn.layers.Dense(10),
    tf.nn.layers.Dense(1)
])
model.compile(
    loss=tf.nn.losses.mse,
    optimizer=tf.nn.optimizers.gradient_descent(),
    metrics=tf.nn.losses.mae
)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
x = tf.range(1, int(1e5))
x = tf.expand_dims(x, axis=1)
%timeit model.fit(x, x + 10, epochs=100, verbose=0)

307 ms ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
